In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from shebaspectral import read_shebaspectral_csv

In [39]:
fili = "C:/Users/apbarret/Documents/data/Sheba2/ShebaSpectral.csv"
date, target, wavelength, spectra = read_shebaspectral_csv(fili)
print type(target)
print type(date)
print type(wavelength)

<type 'list'>
<type 'list'>
<type 'numpy.ndarray'>


Get indices for ponds, ice and leads.  Also put other targets into miscelaneous bin

In [59]:
import re
ispnd = re.compile('pond|mp', re.IGNORECASE)
isice = re.compile('white ice', re.IGNORECASE)
isled = re.compile('lead', re.IGNORECASE)

idxpnd = []
idxice = []
idxled = []
idxmis = []

for i, tgt in list(enumerate(target)):
    if ispnd.search(tgt):
        idxpnd.append(i)
    else:
        if isice.search(tgt):
            idxice.append(i)
        else:
            if isled.search(tgt):
                idxled.append(i)
            else:
                idxmis.append(i)
                
#for ii in idxmis:
#    print target[ii]

Generate new name using target designation: pond, ice, lead, misc; original name and date  

In [66]:
spectra_name = []
for tgt, dt in zip(target,date):
    if ispnd.search(tgt):
        spectra_name.append('POND [%s] %s' % (tgt, dt.strftime('%Y%m%d')))
    else:
        if isice.search(tgt):
            spectra_name.append('ICE [%s] %s' % (tgt, dt.strftime('%Y%m%d')))
        else:
            if isled.search(tgt):
                spectra_name.append('LEAD [%s] %s' % (tgt, dt.strftime('%Y%m%d')))
            else:
                spectra_name.append('MISC [%s] %s' % (tgt, dt.strftime('%Y%m%d')))

# Define an envi header dict

In [80]:
def generate_library_header(spectra, wavelength, target, date, name):
    
    # Get dimensions of spectra array [0]: wavebands, [1]: spectra
    dims = spectra.shape
    
    dict = {}
    dict["description"] = "NSIDC Sea Ice Spectral Library\n Built by Andrew P. Barrett <apbarret@nsidc.org>\n [2016-11-21]"
    dict["samples"] = dims[1]
    dict["lines"] = dims[0]
    dict["bands"] = 1
    dict["header offset"] = 0
    dict["file type"] = "ENVI Spectral Library"
    dict["data type"] = 4
    dict["interleave"] = "bsq"
    dict["sensor type"] = "Spectron Engineering SE-590"          
    dict["byte order"] = 0
    dict["z plot titles"] = "Wavelength [nm], spectral albedo"
    dict["wavelength units"] = "Nanometers"
    dict["band names"] = "NSIDC Sea Ice Spectral Library"
    dict["reflectance scale factor"] = 100.
    dict["original names"] = target
    dict["spectra dates"] = date
    dict["spectra names"] = name
    dict["wavelength"] = wavelength
    
    return dict

Create library files

In [104]:
def make_spectral_library(spectra, wavelength, target, date, name, filo, idx):
    
    tgt = [target[i] for i in idx]
    dt = [date[i] for i in idx]
    nm = [name[i] for i in idx]
    
    spctr = spectra[:,idx].T
    
    from spectral.io.envi import gen_params
    header = generate_library_header(spctr, wavelength, tgt, dt, nm)
    params = gen_params(header)

    # Write data to spectral library class
    from spectral.io.envi import SpectralLibrary
    specLib = SpectralLibrary(spctr, header, params)

    specLib.save(filo)

In [106]:
filo = "C:/Users/apbarret/Documents/data/Sheba2/Sheba.SpectralLibrary.ice"
make_spectral_library(spectra, wavelength, target, date, spectra_name, filo, idxice)

filo = "C:/Users/apbarret/Documents/data/Sheba2/Sheba.SpectralLibrary.pond"
make_spectral_library(spectra, wavelength, target, date, spectra_name, filo, idxpnd)

filo = "C:/Users/apbarret/Documents/data/Sheba2/Sheba.SpectralLibrary.lead"
make_spectral_library(spectra, wavelength, target, date, spectra_name, filo, idxled)



In [241]:
fili = r"C:/Users/apbarret/Documents/data/Sheba2/Sheba.SpectralLibrary.pond.hdr"
pond = envi.open(fili)
nspectra = len(pond.names)
print nspectra

57
